We want to obtain the requeriments of some games, as well as the description.

We'll scraping info from steam.

# Cyberpunk 2077

In [1]:
import requests 
from urllib.request import urlopen
from lxml import etree

In [2]:
# get html from site and write to local file
url = "https://store.steampowered.com/app/1091500/Cyberpunk_2077/"
headers = {'Content-Type': 'text/html',}
response = requests.get(url, headers=headers)
raw_html = response.text
html = etree.HTML(raw_html)


* Look For Description

In [3]:
xpath_for_description = "//div[(@id = 'game_area_description') and not(contains(text(), '\r')) ]/text()"


In [4]:
description_txt = html.xpath(xpath_for_description)

In [5]:
description_txt

[]

* Look for minimum requierments

In [6]:
xpath_for_min_req_strongs = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul//strong/text()"
xpath_for_min_req = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong

In [7]:
min_req_strongs = html.xpath(xpath_for_min_req_strongs)
min_req_txt = html.xpath(xpath_for_min_req)


In [8]:
min_req_strongs

['OS:',
 'Processor:',
 'Memory:',
 'Graphics:',
 'DirectX:',
 'Storage:',
 'Additional Notes:']

In [9]:
min_req_txt

[' Windows 7 or 10',
 ' Intel Core i5-3570K or AMD FX-8310',
 ' 8 GB RAM',
 ' NVIDIA GeForce GTX 780 or AMD Radeon RX 470',
 ' Version 12',
 ' 70 GB available space',
 ' In this game you will encounter a variety of visual effects that may provide seizures or loss of consciousness in a minority of people. If you or someone you know experiences any of the above symptoms while playing, stop and seek medical attention immediately.']

* Look for recommended  requierments

In [10]:
xpath_for_recommended_req_strongs = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul//strong/text()"
xpath_for_recommended_req = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong

In [11]:
recommended_req_strongs = html.xpath(xpath_for_recommended_req_strongs)
recommended_req_txt = html.xpath(xpath_for_recommended_req)

In [12]:
recommended_req_strongs

['OS:',
 'Processor:',
 'Memory:',
 'Graphics:',
 'DirectX:',
 'Storage:',
 'Additional Notes:']

In [13]:
recommended_req_txt

[' Windows 10',
 ' Intel Core i7-4790 or AMD Ryzen 3 3200G',
 ' 12 GB RAM',
 ' GTX 1060 6GB / GTX 1660 Super or Radeon RX 590',
 ' Version 12',
 ' 70 GB available space',
 ' SSD recommended']

In [14]:
merged_list_min = [(min_req_strongs[i], min_req_txt[i]) for i in range(0, len(min_req_strongs))]

In [15]:
merged_list_min

[('OS:', ' Windows 7 or 10'),
 ('Processor:', ' Intel Core i5-3570K or AMD FX-8310'),
 ('Memory:', ' 8 GB RAM'),
 ('Graphics:', ' NVIDIA GeForce GTX 780 or AMD Radeon RX 470'),
 ('DirectX:', ' Version 12'),
 ('Storage:', ' 70 GB available space'),
 ('Additional Notes:',
  ' In this game you will encounter a variety of visual effects that may provide seizures or loss of consciousness in a minority of people. If you or someone you know experiences any of the above symptoms while playing, stop and seek medical attention immediately.')]

In [16]:
import numpy as np 
merged_arr_min =np.array(merged_list_min)

In [17]:
merged_arr_min.shape

(7, 2)

In [18]:
merged_list_rec = [(recommended_req_strongs[i], recommended_req_txt[i]) for i in range(0, len(recommended_req_strongs))  ]
merged_arr_recommended = np.array(merged_list_rec)

In [19]:
merged_arr_recommended

array([['OS:', ' Windows 10'],
       ['Processor:', ' Intel Core i7-4790 or AMD Ryzen 3 3200G'],
       ['Memory:', ' 12 GB RAM'],
       ['Graphics:', ' GTX 1060 6GB / GTX 1660 Super or Radeon RX 590'],
       ['DirectX:', ' Version 12'],
       ['Storage:', ' 70 GB available space'],
       ['Additional Notes:', ' SSD recommended']], dtype='<U47')

In [20]:
!pip install selenium

     |████████████████████████████████| 911kB 5.6MB/s 


In [21]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 81.0 MB of archives.
After this operation, 273 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 87.0.4280.66-0ubuntu0.18.04.1 [1,122 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 87.0.4280.66-0ubuntu0.18.04.1 [71.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 87.0.428

In [22]:
import sys

In [23]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.firefox import GeckoDriverManager

In [25]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [26]:
def get_page_link_from_steam(game):
  print(f"Getting page for {game} ..."  )
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get("https://store.steampowered.com/")
  assert "Steam" in driver.title #assert is for debugging, if the condition is false, AssertionError is raised. 
  #Here we are using it to confirm that the title has "Steam".
  elem = driver.find_element_by_id("store_nav_search_term")
  elem.clear()
  elem.send_keys(game)
  elem.send_keys(Keys.ENTER)

 # print(driver.page_source)
 
  driver.find_element_by_xpath("//div[(@id = 'search_resultsRows')]/a[1]").click()
  source_str =  str(driver.page_source)
   #If not appropiate for all ages
  #print(source_str)
  if "View Page" in source_str:
    print("entered")

    #Specify the age
    element = driver.find_element_by_xpath("//select[@name='ageYear']")
    all_options = element.find_elements_by_tag_name("option")
    for option in all_options:
      #print("Value is: %s" % option.get_attribute("value"))
      if  option.get_attribute("value") == '1900':
        print("clicked")
        option.click()
        

    driver.find_element_by_xpath("//div[(@class = 'main_content_ctn')]/div[@class = 'agegate_text_container btns']/a[1]").click() 
    time.sleep(1) #  wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'tt-open')))
  return driver.current_url
  


* Once we have the link that we need, let's scrap the page as we did it before.


In [27]:
def scrap_page_steam(link_path):
  print(f"Scraping {link_path} ..."  )
  headers = {'Content-Type': 'text/html',}
  response = requests.get(link_path, headers=headers)
  raw_html = response.text
  html = etree.HTML(raw_html)

#Get description
  xpath_for_description = "//div[(@id = 'game_area_description')]/text()"
  description_txt = html.xpath(xpath_for_description)

#Get Minimum Requirements
  xpath_for_min_req_strongs = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul//strong/text()"
  xpath_for_min_req = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong
  min_req_strongs = html.xpath(xpath_for_min_req_strongs)
  min_req_txt = html.xpath(xpath_for_min_req)

#Get Recommended Requirements
  xpath_for_recommended_req_strongs = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul//strong/text()"
  xpath_for_recommended_req = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong
  recommended_req_strongs = html.xpath(xpath_for_recommended_req_strongs)
  recommended_req_txt = html.xpath(xpath_for_recommended_req)

#Merge strongs and text in the same array
  merged_list_min = [(min_req_strongs[i], min_req_txt[i]) for i in range(0, len(min_req_strongs))]
  merged_arr_min =np.array(merged_list_min)

  merged_list_rec = [(recommended_req_strongs[i], recommended_req_txt[i]) for i in range(0, len(recommended_req_strongs)) ]
  merged_arr_recommended = np.array(merged_list_rec)

  return description_txt, merged_arr_min, merged_arr_recommended

In [28]:
page_to_scrap = get_page_link_from_steam('hellblade')
page_to_scrap


Getting page for hellblade ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


'https://store.steampowered.com/app/414340/Hellblade_Senuas_Sacrifice/'

In [29]:
page_to_scrap

'https://store.steampowered.com/app/414340/Hellblade_Senuas_Sacrifice/'

In [30]:
descr, min, recommended = scrap_page_steam(page_to_scrap)

Scraping https://store.steampowered.com/app/414340/Hellblade_Senuas_Sacrifice/ ...


In [31]:
descr

['\r\n\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t',
 'From the makers of Heavenly Sword, Enslaved: Odyssey to the West, and DmC: Devil May Cry, comes a warrior’s brutal journey into myth and madness.',
 'Set in the Viking age, a broken Celtic warrior embarks on a haunting vision quest into Viking Hell to fight for the soul of her dead lover. ',
 'Created in collaboration with neuroscientists and people who experience psychosis, Hellblade: Senua’s Sacrifice will pull you deep into Senua’s mind.',
 '\t\t\t\t\t\t']

In [32]:
min

array([], dtype=float64)

In [33]:
recommended

array([], dtype=float64)

* Let's scrap pcgamebenchmark.com

In [34]:
def get_game_link_pcgbm(game):
  print(f"Getting page for {game} ..."  )
  
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get("https://www.pcgamebenchmark.com/")
  assert "PCGameBenchmark" in driver.title #assert is for debugging, if the condition is false, AssertionError is raised. 
  #Here we are using it to confirm that the title has "Steam".
  wait = WebDriverWait(driver,10)
  elem = driver.find_element_by_xpath("//form[(@id = 'search')]/span/input[@name = 'q']")
  elem.clear()
  elem.send_keys(game)

  wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'tt-open')))
  
  elem.send_keys(Keys.ENTER)
  print(driver.current_url)

  
  return driver.current_url
  


In [67]:
def scrap_from_pcgbm(game_url):
  print(f"Scraping {game_url} ..."  )
  headers = {'Content-Type': 'text/html',}
  response = requests.get(game_url, headers=headers)
  raw_html = response.text
  html = etree.HTML(raw_html)

#Get description
  xpath_for_description = "//div[(@id = 'details-meta')]/div[@class = 'description']/p/text()"
  description_txt = html.xpath(xpath_for_description)

#Get Minimum Requirements
  xpath_for_min_req_strongs = "//div[(@class = 'row requirements')]/div[1]/ul/li/strong/text() | //div[(@class = 'requirements')]/ul/li/strong/text()"
  xpath_for_min_req = "//div[(@class = 'row requirements')]/div[1]/ul/li[strong]/text() | //div[(@class = 'requirements')]/ul/li[strong]/text()" #With [] we say to look for those li with strong
  min_req_strongs = html.xpath(xpath_for_min_req_strongs)
  min_req_txt = html.xpath(xpath_for_min_req)

#Get Recommended Requirements
  xpath_for_recommended_req_strongs = "//div[(@class = 'row requirements')]/div[2]/ul/li/strong/text()"
  xpath_for_recommended_req = "//div[(@class = 'row requirements')]/div[2]/ul/li[strong]/text()"#With [] we say to look for those li with strong
  recommended_req_strongs = html.xpath(xpath_for_recommended_req_strongs)
  recommended_req_txt = html.xpath(xpath_for_recommended_req)

#Merge strongs and text in the same array
  merged_list_min = [(min_req_strongs[i], min_req_txt[i]) for i in range(0, len(min_req_strongs))]
  merged_arr_min =np.array(merged_list_min)

  merged_list_rec = [(recommended_req_strongs[i], recommended_req_txt[i]) for i in range(0, len(recommended_req_strongs)) ]
  merged_arr_recommended = np.array(merged_list_rec)


  return description_txt, merged_arr_min, merged_arr_recommended

In [63]:
pcgbm_link = get_game_link_pcgbm('among us')

Getting page for among us ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


https://www.pcgamebenchmark.com/among-us-system-requirements


In [68]:
description_txt_pcgbm, min_req_pcgbm, recommended_req_pcgbm = scrap_from_pcgbm(pcgbm_link)

Scraping https://www.pcgamebenchmark.com/among-us-system-requirements ...


In [69]:
description_txt_pcgbm

['An online and local party game of teamwork and betrayal for 4-10 players...in space!']

In [70]:
min_req_pcgbm

array([['Memory:', '2 GB'],
       ['Graphics Card:', 'NVIDIA GeForce GT 610'],
       ['Among Us CPU:', 'Intel Core i3-2120'],
       ['File Size:', '1 GB'],
       ['OS:', 'Windows 10'],
       ['Memory:', '1 GB'],
       ['Graphics Card:', 'NVIDIA GeForce 510'],
       ['CPU:', 'Intel Pentium 4 2.00GHz'],
       ['Among Us File Size:', '250 MB'],
       ['OS:', 'Windows 7 SP1+']], dtype='<U23')

In [61]:
recommended_req_pcgbm

array([], dtype=float64)